In [ ]:
#The Python code starts here until the R code starts




import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LogisticRegression , LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SequentialFeatureSelector
!pip install asgl
import asgl
import time

In [ ]:
#load in data
dat=pd.read_csv('fifa18_clean.csv')

In [ ]:

#remove non numerical columns
dat=dat.drop(['Name','Photo','Nationality','Flag','Club','Club Logo','Wage (€)','Preferred Positions','ID'],axis=1)
dat=dat.dropna()
dat.head()

,Value (€),Age,Overall,Potential,Special,Acceleration,Aggression,Agility,Balance,Ball control,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,95500000,32,94,94,2228,89.0,63.0,89.0,63.0,93.0,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,105000000,30,93,93,2154,92.0,48.0,90.0,95.0,95.0,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,123000000,25,92,94,2100,94.0,56.0,96.0,82.0,95.0,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,97000000,30,92,92,2291,88.0,78.0,86.0,60.0,91.0,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
5,92000000,28,91,91,2143,79.0,80.0,78.0,80.0,89.0,...,58.0,57.0,78.0,62.0,87.0,82.0,88.0,84.0,61.0,88.0


In [ ]:
#split the dataframe into variables and target
targ=dat['Overall']
var=dat.drop('Overall',axis=1)


targ.to_csv(r'/content/target.csv',index=False,header=True)
var.to_csv(r'/content/var.csv',index=False,header=True)

In [ ]:
#perform LASSO

ts=time.time()
las=SelectFromModel(Lasso(alpha=1,random_state=10))
las.fit(var,targ)
a=las.get_support()
lv=var.drop(np.array(var.columns[(las.estimator_.coef_==0).ravel().tolist()]),axis=1)
tss=time.time()
print(tss-ts)

0.11608600616455078


In [ ]:
lv.head()

,Value (€),Age,Potential,Special,Acceleration,Balance,Ball control,Composure,Free kick accuracy,Heading accuracy,Long shots,Positioning,Reactions,Short passing,Sprint speed,Stamina,Strength,Vision
0,95500000,32,94,2228,89.0,63.0,93.0,95.0,76.0,88.0,92.0,95.0,96.0,83.0,91.0,92.0,80.0,85.0
1,105000000,30,93,2154,92.0,95.0,95.0,96.0,90.0,71.0,88.0,93.0,95.0,88.0,87.0,73.0,59.0,90.0
2,123000000,25,94,2100,94.0,82.0,95.0,92.0,84.0,62.0,77.0,90.0,88.0,81.0,90.0,78.0,53.0,80.0
3,97000000,30,92,2291,88.0,60.0,91.0,83.0,84.0,77.0,86.0,92.0,93.0,83.0,77.0,89.0,80.0,84.0
5,92000000,28,91,2143,79.0,80.0,89.0,87.0,84.0,85.0,83.0,91.0,91.0,83.0,83.0,79.0,84.0,78.0


In [ ]:
#perform adaptive lasso


ts=time.time()
alasso = asgl.TVT(model='lm', penalization='alasso', lambda1= .001*np.arange(0, 10, 0.1),parallel=True,
                      weight_technique='lasso', error_type='MSE', random_state=1)
alasso_result = alasso.train_validate_test(x=var.to_numpy(), y=targ.to_numpy())
print(time.time()-ts)

/usr/local/lib/python3.8/dist-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


15.491676330566406


In [ ]:
alv=var.columns[np.nonzero(alasso_result['optimal_betas'][1:])]
alv2=var[alv]

In [ ]:
#Lasso + forward elimination

ts=time.time()
las=SelectFromModel(Lasso(alpha=.2,random_state=10))
las.fit(var,targ)
a=las.get_support()
var2=var.drop(np.array(var.columns[(las.estimator_.coef_==0).ravel().tolist()]),axis=1)

vsel=var2.iloc[:,0:1]
model=LinearRegression().fit(vsel,targ)
vselscore=(model.score(vsel,targ))
for i in np.arange(0,len(var2.columns)):
  v=var2.iloc[:,i:i+1]
  model=LinearRegression().fit(v,targ)
  vscore=model.score(v,targ)
  if(vscore>vselscore):
    vselscore=vscore
    vsel=v
var2=var2.drop(vsel,axis=1)
vog=vselscore
vt=vsel

j=0
k=0
while j==0:
  vselscore=0
  for i in np.arange(0,len(var2.columns)):
    v1=var2.iloc[:,i:i+1]
    v=pd.concat([vt,v1],axis=1)
    model=LinearRegression().fit(v,targ)
    vscore=model.score(v,targ)
    if(vscore>vselscore):
      vselscore=vscore
      vsel=v1
  


  if(-vog+vselscore <= .001):
    j=1
  
  if(-vog+vselscore>.001):
    vt=pd.concat([vt,vsel],axis=1)
    vog=vselscore
    var2=var2.drop(vsel,axis=1)
  print(vog)
  k=k+1
  if(k>40):
    j=1

print(time.time()-ts)

0.7826647641019246
0.8906682344248837
0.9031338121742389
0.909382788072085
0.9147208335562252
0.9171800828479997
0.9198402609045595
0.9220698482222343
0.9239793025917172
0.9266109722306634
0.9266109722306634
4.020595073699951


In [ ]:
vt

,Reactions,Potential,Age,Special,Heading accuracy,Value (€),Sprint speed,Composure,Strength,Ball control,Positioning
0,96.0,94,32,2228,88.0,95500000,91.0,95.0,80.0,93.0,95.0
1,95.0,93,30,2154,71.0,105000000,87.0,96.0,59.0,95.0,93.0
2,88.0,94,25,2100,62.0,123000000,90.0,92.0,53.0,95.0,90.0
3,93.0,92,30,2291,77.0,97000000,77.0,83.0,80.0,91.0,92.0
5,91.0,91,28,2143,85.0,92000000,83.0,87.0,84.0,89.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...
17974,48.0,56,18,1105,43.0,40000,60.0,37.0,42.0,34.0,27.0
17975,47.0,46,28,1174,38.0,30000,22.0,42.0,37.0,46.0,39.0
17978,54.0,61,17,1194,47.0,60000,51.0,50.0,32.0,41.0,46.0
17979,49.0,64,18,1302,40.0,60000,58.0,45.0,46.0,43.0,37.0


In [ ]:
!pip install abess
from abess.linear import LinearRegression as LinReg

In [ ]:
#perform ABESS
ts=time.time()
mod=LinReg()
mod.fit(var.to_numpy(),targ.to_numpy())
print(time.time()-ts)

1.6084558963775635


In [ ]:
absv=var[var.columns[np.nonzero(mod.coef_)]]

In [ ]:
#take data from R notebook to get L0Learn's dataframe
lol=np.array([1,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
lol=var[var.columns[np.nonzero(lol)]]


In [ ]:
#create 1 big array of all columns
t=np.append(lv.columns,alv)
t=np.append(t,vt.columns)
t=np.append(t,absv.columns)
t=np.append(t,lol.columns)

In [ ]:
#figure out which ones share values in other dataframes and which is in each
np.unique(t,return_counts=True)

(array(['Acceleration', 'Age', 'Aggression', 'Agility', 'Balance',
        'Ball control', 'Composure', 'Crossing', 'Curve', 'Dribbling',
        'Finishing', 'Free kick accuracy', 'GK handling', 'GK kicking',
        'GK reflexes', 'Heading accuracy', 'Interceptions', 'Jumping',
        'LS', 'LWB', 'Long shots', 'Marking', 'Penalties', 'Positioning',
        'Potential', 'RS', 'RWB', 'Reactions', 'ST', 'Short passing',
        'Shot power', 'Sliding tackle', 'Special', 'Sprint speed',
        'Stamina', 'Standing tackle', 'Strength', 'Value (€)', 'Vision',
        'Volleys'], dtype=object),
 array([4, 5, 1, 1, 2, 5, 5, 1, 1, 1, 1, 2, 1, 2, 1, 5, 1, 1, 1, 1, 2, 3,
        1, 5, 5, 1, 1, 5, 1, 4, 1, 1, 3, 5, 4, 1, 5, 4, 3, 1]))

In [ ]:
absv.columns

Index(['Value (€)', 'Age', 'Potential', 'Acceleration', 'Balance',
       'Ball control', 'Composure', 'Crossing', 'Finishing', 'GK kicking',
       'Heading accuracy', 'Marking', 'Positioning', 'Reactions',
       'Short passing', 'Sprint speed', 'Stamina', 'Strength', 'Vision'],
      dtype='object')

In [ ]:
#R2 for lasso
m1=LinearRegression()
m1.fit(lv,targ)
m1.score(lv,targ)

0.929766220747539

In [ ]:
#R2 for adaptive lasso
m2=LinearRegression()
m2.fit(alv2,targ)
m2.score(alv2,targ)

0.9258953264980461

In [ ]:
#R2 for ABBES
m3=LinearRegression()
m3.fit(absv,targ)
m3.score(absv,targ)

0.9301295280408212

In [ ]:
#R2 for lasso + forward elimination
m4=LinearRegression()
m4.fit(vt,targ)
m4.score(vt,targ)

0.9266109722306634

In [ ]:
#R2 for L0Learn
m5=LinearRegression()
m5.fit(lol,targ)
m5.score(lol,targ)

0.9287436136848077